<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Design" data-toc-modified-id="Design-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Design</a></span></li><li><span><a href="#Charts" data-toc-modified-id="Charts-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Charts</a></span><ul class="toc-item"><li><span><a href="#Data" data-toc-modified-id="Data-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Data</a></span></li><li><span><a href="#Flere-charts-i-samme-figur" data-toc-modified-id="Flere-charts-i-samme-figur-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Flere charts i samme figur</a></span><ul class="toc-item"><li><span><a href="#Compound" data-toc-modified-id="Compound-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Compound</a></span></li><li><span><a href="#Concatination" data-toc-modified-id="Concatination-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>Concatination</a></span></li><li><span><a href="#repeat" data-toc-modified-id="repeat-2.2.3"><span class="toc-item-num">2.2.3&nbsp;&nbsp;</span>repeat</a></span></li></ul></li></ul></li><li><span><a href="#Marks" data-toc-modified-id="Marks-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Marks</a></span><ul class="toc-item"><li><span><a href="#punkter" data-toc-modified-id="punkter-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>punkter</a></span></li><li><span><a href="#Rektangler" data-toc-modified-id="Rektangler-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Rektangler</a></span></li><li><span><a href="#Linjer" data-toc-modified-id="Linjer-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Linjer</a></span></li><li><span><a href="#Tekst" data-toc-modified-id="Tekst-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Tekst</a></span></li><li><span><a href="#Annet" data-toc-modified-id="Annet-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Annet</a></span></li></ul></li><li><span><a href="#Encoding" data-toc-modified-id="Encoding-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Encoding</a></span><ul class="toc-item"><li><span><a href="#Spesifisere-encoding" data-toc-modified-id="Spesifisere-encoding-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Spesifisere encoding</a></span></li><li><span><a href="#Ulike-typer-encoding" data-toc-modified-id="Ulike-typer-encoding-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Ulike typer encoding</a></span><ul class="toc-item"><li><span><a href="#Posisjon" data-toc-modified-id="Posisjon-4.2.1"><span class="toc-item-num">4.2.1&nbsp;&nbsp;</span>Posisjon</a></span></li><li><span><a href="#Farge" data-toc-modified-id="Farge-4.2.2"><span class="toc-item-num">4.2.2&nbsp;&nbsp;</span>Farge</a></span></li><li><span><a href="#Andre-estetiske-egenskaper" data-toc-modified-id="Andre-estetiske-egenskaper-4.2.3"><span class="toc-item-num">4.2.3&nbsp;&nbsp;</span>Andre estetiske egenskaper</a></span></li><li><span><a href="#Annet" data-toc-modified-id="Annet-4.2.4"><span class="toc-item-num">4.2.4&nbsp;&nbsp;</span>Annet</a></span></li></ul></li><li><span><a href="#Transformasjoner" data-toc-modified-id="Transformasjoner-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Transformasjoner</a></span></li><li><span><a href="#Aggregering" data-toc-modified-id="Aggregering-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Aggregering</a></span></li><li><span><a href="#binning" data-toc-modified-id="binning-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>binning</a></span><ul class="toc-item"><li><span><a href="#Histogram" data-toc-modified-id="Histogram-4.5.1"><span class="toc-item-num">4.5.1&nbsp;&nbsp;</span>Histogram</a></span></li><li><span><a href="#Fargekart" data-toc-modified-id="Fargekart-4.5.2"><span class="toc-item-num">4.5.2&nbsp;&nbsp;</span>Fargekart</a></span></li><li><span><a href="#Binning-med-tid" data-toc-modified-id="Binning-med-tid-4.5.3"><span class="toc-item-num">4.5.3&nbsp;&nbsp;</span>Binning med tid</a></span></li></ul></li><li><span><a href="#facets" data-toc-modified-id="facets-4.6"><span class="toc-item-num">4.6&nbsp;&nbsp;</span>facets</a></span></li></ul></li><li><span><a href="#Seleksjon" data-toc-modified-id="Seleksjon-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Seleksjon</a></span><ul class="toc-item"><li><span><a href="#Typer-seleksjon" data-toc-modified-id="Typer-seleksjon-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Typer seleksjon</a></span></li><li><span><a href="#Conditional-encodings" data-toc-modified-id="Conditional-encodings-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Conditional encodings</a></span></li><li><span><a href="#Filter" data-toc-modified-id="Filter-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Filter</a></span></li><li><span><a href="#Scales" data-toc-modified-id="Scales-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Scales</a></span></li><li><span><a href="#Domain" data-toc-modified-id="Domain-5.5"><span class="toc-item-num">5.5&nbsp;&nbsp;</span>Domain</a></span></li></ul></li><li><span><a href="#Configuration" data-toc-modified-id="Configuration-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Configuration</a></span><ul class="toc-item"><li><span><a href="#Properties" data-toc-modified-id="Properties-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Properties</a></span></li><li><span><a href="#Legend" data-toc-modified-id="Legend-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Legend</a></span></li></ul></li></ul></div>

### Design

High level api som generer vega-lite spesifikasjon. Med andre ord så lager det json som kan bli renderet av frontend. 

Bygger på grammar of graphics, så er litt analog til ggplot

Kan kjøre flere encodes, kan være måte å spare kode

Må generelt bruke masse funksjoner (ikke strings direkte som argument) fordi den prøver å inferre alle strings som kolonnenavn

```python
alt.Chart(data).mark_point().encode(
    encoding_1='column_1',
    encoding_2=alt.function('column_2', *args)
    ...
).properties( # lurer på om jeg kan sette dette som argumneter i mark-funksjonen
    height,
    width,
    ...
).configure_axis(
    labelFontSize,
    titleFontSize
).configure_legend(
    orient
).configure_point(
    size
)
```

### Charts

Objekt som lage json dictionary som kan renderes av vega-lite library i js

compound med + 

#### Data

Merk at encoder data i Chart (output til json). Har derfor lagt inn begrensning om maks 5000 observasjoner. Hvis jobber med større datasett så bør jeg aggregere før jeg sender inn inn i Chart

Merk også at det er designet for å jobbe med url-basert data der det ikke går gjennom pandas. Må da spesifisere variabeltype eksplisitt.

Merk at den bare jobber med kolonner; ikke indeks. Må eksplisitt legge index til som kolonne.

#### Flere charts i samme figur 

Litt usikker på terminologi. Også litt ukomofortabel med at det ikke er eksplisitt grid; føler jeg mangler litt kontroll på layout.

##### Compound

Legge charts oppå hverandre. Har to måter

```python
alt.layer(chart1, chart2)
chart1 + chart
```
Hvis vi f.eks. vil ha samme encoding men to forskjellige markers (linje og punkt), så kan vi utnytte fleksibel api for å gjenbruke kode
```python
base = alt.Chart(data).encode(x,y)
base.mark_line() + base.mark_point()
```

##### Concatination

Kan lage figur med flere subplots(?) ved å enten legge charts oppå eller ved siden av hverandre

```python
# ved siden
alt.hconcat(chart1, chart2)
chart1 | chart2

# oppå
alt.vconcat(chart1, chart2)
chart1 & chart2
```

##### repeat

Kan bruke repeat-funksjonen for å lage parvis scatterplot..
```python
fields = ['petalLength', 'petalWidth', 'sepalLength', 'sepalWidth']

alt.Chart(iris).mark_point().encode(
    alt.X(alt.repeat("column"), type='quantitative'),
    alt.Y(alt.repeat("row"), type='quantitative'),
    color='species'
).properties(
    width=150,
    height=150
).repeat(
    row=fields,
    column=fields
)
```

### Marks

Tilsvarer geom i ggplot2, de ulike geometriske objektene i plottet.

#### punkter

```python
mark_point()
mark_circle()
mark_square()
mark_tick()
```

#### Rektangler

```python
mark_bar()
mark_rect()
```

#### Linjer

```python
mark_line()
```

#### Tekst

```python
mark_text().encode(
    x,
    y,
    text
)
```
kan kombineres med annen Chart og legge opp for å f.eks. få tall på toppen av bar plot

#### Annet

```python
mark_boxplot()
```

### Encoding

Spesifiserer mapping fra variabel til Marks ulike estetiske egenskaper

Tolker scale (innhold i mapping mellom dataspace og estetikkspace) ut fra datatype
1. Quantitative (Q)
2. Ordinal (O)
3. Nominal (N)
4. Tid (T)

Skal sjå litt på ulike typer encodings

#### Spesifisere encoding

Det er fleksibel api der vi kan spesifisere på ulike måter. En mulighet er å bruke shorthand, som er en string som angir field (kolonnenavn) og eventuelt datatype og aggregate. Tre ulike måter å spesifisere samme encoding:
```python
chart.encode(
    x='mean(colname):Q')

chart.encode(
    alt.X(shorthand='mean(colname):Q'))

chart.encode(
    alt.X(field='colname',aggregate='mean', type='quantitative))

```
de ulike funksjonene har flere argument enn de som kan angis gjennom shorthand, så får mer fleksibilitet gjennom å eksplisitt bruke funksjonene

#### Ulike typer encoding

##### Posisjon

Hvordan jeg ønsker å håndtere dette avhenger litt av om direkte tolkning av plassering som numerisk verdi eller kun ordinal/nominal distinksjon mellom objekt

```python
chart.encode(
    alt.X('column',
        scale=alt.Scale(domain=(5, 20)), # kan sette axis limits
        axis=alt.Axis(...) # må i hvertfall spesifisere title (axis label)
)          
```

```python
alt.Axis(
    title, # axis label
    labels,
    ticks,
    format='%-d/%-m/%Y' # kontrollere hvis tid
)
```

##### Farge

Får mer kontroll over fargevalg ved å kombinere alt.Color() med et Scale-objekt


```python

custom_scale = alt.Scale(domain=['val0', 'val1',...], range=['color0', 'color1',...])
builtin_scale = alt.Scale(scheme='dark2)

chart.encode(
    alt.Color('column:N', scale=custom_scale # eller builtin_scale .. 
)
```

##### Andre estetiske egenskaper

- opacity
- shape
- size

##### Annet

- tooltip

#### Transformasjoner

Kan spesifiserer transformasjon av variabel,
```python
chart.encode(
    attribute='mean(colname)'
)
```
Kan også bruke
```python
chart.encode(
    x='colname',
    y='count()
)
```
for at plassering i y-retning avhenger av antall observasjoner som encodet på gitt plassering i x-retning

#### Aggregering

Kan gjøre eksplisitt med transform_ funksjon

```python
chart.encode(
    x='colname',
    y='agg_field_name' # eks 'mean_of_colname2'
).transform_aggregate(
    agg_field_name='mean(colname2)', #  tar variabelnavn som ble angitt i encoding som argument ... hmhm
    groupby=['colname'] # må være array
)
```

Kan eventuelt gjøre det gjennom encoding i stedet for å transformere ex-post
```python
chart.encode(
    x='colname',
    y='mean(colname2)' # skjønner at groupby colname ..
)
```


#### binning

##### Histogram

Partisjonering av utfallsmengden til variabel... behandle observasjonene som faller inn i hver av delmengdene som ulike grupper. Blir en måte å konstruere slags diskret variabel som vi deretter kan kjøre groupby på og finne aggegert mål på hver gruppe (feks sum). På denne måten kan vi (blant annet?) lage histogram. Merk at vi må representere bredde på intervall i delmengdene, så bredde på bar er ikke lenger vilkårlig (i motsetning til barplot som egentlig kun representer et punkt (x,y))

Kan enten spesifisere at bin=True eller initialisere Bin-objekt som gir oss flere arguemnt
```python
chart.encode(
    x=alt.X('Miles_per_Gallon', bin=alt.Bin(maxbins=20)),
    y='count()'
)
```
Bin har litt ulike argumenter, men i praksis vil bare bruke maxbins siden den prøver å utlede en god partisjonering (splitte på heltall)


##### Fargekart

Tilsvarer egentlig histogram, bortsett fra at vi bruker farge til å indikere verdi i hver bin i stedet for lengde på bar i y-retning.
```python
alt.Chart(data).mark_rect().encode(
    alt.X('col_x', bin=True),
    alt.Y('col_y', bin=True),
    color='count():O'
)
```

##### Binning med tid

Har ulike funksjoner for å binne data med tidsdimensjon
```python
chart.encode(
    x='day(time_var):O', # alternativer: hours, month, year (mer? sikkert minutter og sekunder..)
    y='mean(some_var)'
)
```

#### facets

Kan også bruke encoding til å legge ut plot på subset av observasjoner i grid
- row
- column

### Seleksjon

Workflow:
1. Initialisere seleksjonsobjekt
2. Legge til seleksjonsobjekt til chart med .add_selection()
3. Bruke alt.condition() på attribut i encode-funksjonen for å gjøre ting avhengig av seleksjon

#### Typer seleksjon

Har tre typer seleksjonsobjeter

```python
alt.selection_interval(
    encodings, # ['x'] hvis kun intervall på x-akse. Default er ['x','y'] som gir rektangel
    bind # 'scales' gir basically samme som interact.. kan konstruere zoom gjennom bind på intervall..
)    
```

```python
alt.selection_single(
    on, # 'click', 'mouseover'
    nearest, # bør være True hvis on='mouseover'
    encodings # velge alle marks med samme egenskap av encoding, eks ['color']
)
```

```python
alt.selection_multi(

)
```

#### Conditional encodings

```python
alt.condition(predicate=selection_obj,
              if_true=mapping, # i praksis kolonnenavn.. hva den mapper fra.
              if_false=some_value) # eller kanskje annen mapping
```

Bruker én mapping fra data til visuell egenskap for observasjon/objekt som er valgt og annen mapping for resten. Bruker condition-funksjon som argument for egenskap i encode, eks:
```python
chart.encode(x,y,
    color=alt.condition(interval, 'species', alt.value('lightgray'))
).add_selection(interval)
```

#### Filter

Bruker
```python
chart.transform_filter(selection_obj)
```
for å subsette observasjoner som brukes til å lage marks i chart. Synes det er litt problematisk at axis blir oppdater automatisk slik at akkurat får inn alle observasjonene.. bør finne en måte å gjøre dette statisk.

#### Scales

#### Domain

Kan bruke seleksjon i én chart til å definere domain i annen chart slik at vi i praksis zoomer inn på angitt intervall.

```python
chart = base.encode(x=alt.X('date:T', scale=alt.Scale(domain=interval.ref())))
view = base.add_selection(interval)
chart & view                    
```
Har samme x encoding, men på chart er domenet til x angitt av interval som vi bestemmer i view

### Configuration

Har tre nivåer der vi kan spesifisere egenskaper ved utseendet til chart

```python
# 'red'>'blue'>'green' fordi encode>local>global
alt.Chart(data).mark_point(color='blue').encode( 
    x,y,
    color=alt.value('red') # må bruke alt.value for at den ikke skal tro det er kolonnenavn
).configure_point(color='green') 
```

#### Properties

Kan sette en del egenskaper gjennom .properties metoden ..

#### Legend

Kontrollerer gjennom encoding..

```python
chart.encode(
    ...,
    alt.Color('colname', legend=alt.Legend(...)) # encoding som generer legend
)
```

```python
alt.Legend(
    title, # None hvis supress
    titleFontSize,
    labelFontSize
)
```